In [44]:
# lets make a decision tree neat cool awesome I love it
# custom modules for data prep:
import acquire as a
import prepare as p
# tabular manipulation
import numpy as np
import pandas as pd
# ML stuff:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier, \
export_text, plot_tree
# import our Random Forest model from sklearn:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

What is your baseline prediction? 

What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). 

When you make those predictions, what is your accuracy? This is your baseline accuracy.

Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [13]:
df = a.get_titanic_data()
df

this file exists, reading from csv


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,NaN,Southampton,1
887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.4500,S,Third,NaN,Southampton,0
889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


In [14]:
df = p.clean_titanic(df)
df

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone
0,0,0,3,male,1,0,7.2500,Southampton,0
1,1,1,1,female,1,0,71.2833,Cherbourg,0
2,2,1,3,female,0,0,7.9250,Southampton,1
3,3,1,1,female,1,0,53.1000,Southampton,0
4,4,0,3,male,0,0,8.0500,Southampton,1
...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,0,0,13.0000,Southampton,1
887,887,1,1,female,0,0,30.0000,Southampton,1
888,888,0,3,female,1,2,23.4500,Southampton,0
889,889,1,1,male,0,0,30.0000,Cherbourg,1


In [15]:
train_df, val_df, test_df = p.splitting_data(df, 'survived')

In [16]:
def preprocess_titanic(train_df, val_df, test_df):
    '''
    preprocess_titanic will take in three pandas dataframes
    of our titanic data, expected as cleaned versions of this 
    titanic data set (see documentation on acquire.py and prepare.py)
    
    output:
    encoded, ML-ready versions of our clean data, with 
    columns sex and embark_town encoded in the one-hot fashion
    return: (pd.DataFrame, pd.DataFrame, pd.DataFrame)
    '''
    # with a looping structure:
    # for df in [train_df, val_df, test_df]:
    #     df.drop(blah blah blah)
    #     df['pclass'] = df['pclass'].astype(int)
    train_df = train_df.drop(columns='passenger_id')
    train_df['pclass'] = train_df['pclass'].astype(int)
    val_df = val_df.drop(columns='passenger_id')
    val_df['pclass'] = val_df['pclass'].astype(int)
    test_df = test_df.drop(columns='passenger_id')
    test_df['pclass'] = test_df['pclass'].astype(int)
    encoding_var = ['sex', 'embark_town']
    encoded_dfs = []
    for df in [train_df, val_df, test_df]:
        df_encoded_cats = pd.get_dummies(
            df[['embark_town', 'sex']],
              drop_first=True).astype(int)
        encoded_dfs.append(pd.concat(
            [df,
            df_encoded_cats],
            axis=1).drop(columns=['sex', 'embark_town']))
    return encoded_dfs

In [17]:
train_encoded, val_encoded, test_encoded = preprocess_titanic(train_df, val_df, test_df)

In [81]:
# model ready dataset:
# all nums, no Objects:
train_encoded.head()

,survived,pclass,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
776,0,3,0,0,7.7500,1,1,0,1
829,1,1,0,0,80.0000,1,0,1,0
215,1,1,1,0,113.2750,0,0,0,0
258,1,1,0,0,512.3292,1,0,0,0
129,0,3,0,0,6.9750,1,0,1,1


In [82]:
train_encoded.dtypes

survived                     int64
pclass                       int64
sibsp                        int64
parch                        int64
fare                       float64
alone                        int64
embark_town_Queenstown       int64
embark_town_Southampton      int64
sex_male                     int64
dtype: object

In [73]:
X_train, y_train = train_encoded.drop(columns='survived'), \
train_encoded.survived

In [74]:
X_val, y_val = val_encoded.drop(columns='survived'), \
val_encoded.survived

In [75]:
X_test, y_test = test_encoded.drop(columns='survived'), \
test_encoded.survived

In [76]:
X_train.columns

Index(['pclass', 'sibsp', 'parch', 'fare', 'alone', 'embark_town_Queenstown',
       'embark_town_Southampton', 'sex_male'],
      dtype='object')

In [21]:
y_train.head()

776    0
829    1
215    1
258    1
129    0
Name: survived, dtype: int64

In [22]:
# creating the model!
# SKLEARN PROCESS:
# invoke/create a variable that holds an object:
# make the thing:
clf = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)

In [23]:
clf

DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)

In [24]:
# fit the thing
# NOTE: ONLY DO THIS STEP ON TRAIN.
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)

In [25]:
# use the thing!
# now that clf is fit, use it to predict!
clf.predict(X_train)[:10]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

In [26]:
# before we proceed, lets create a baseline prediction:
# we do this by making a blanket prediction of the majority class
train_encoded.survived.value_counts()

survived
0    329
1    205
Name: count, dtype: int64

In [27]:
# lets make a predictions dataframe to hold
# y_true: the actual values of our target
# baseline: the blanket all-zero survival prediction
# a new column for any model we create
# so we can compare them all to eachother

In [28]:
y_pred = pd.DataFrame({
    'y_true': y_train.values,
    'baseline':0,
    'dt_one': clf.predict(X_train)
}, index=train_encoded.index)

In [35]:
y_pred.head(2)

,y_true,baseline,dt_one
776,0,0,0
829,1,0,1


In [ ]:
# What is your baseline prediction? My baseline prediction is that more people die.

In [30]:
# now that we have these organized here:
# lets compare what the baseline is doing
# and what the model is doing:
# we pulled in accuracy score from sklearn:
baseline_acc = accuracy_score(y_pred.y_true, y_pred.baseline)

In [31]:
baseline_acc

0.6161048689138576

In [32]:
y_val_pred = pd.DataFrame({
    'y_true': val_encoded.survived,
    'baseline':0,
    'dt_one': clf.predict(val_encoded.drop(columns='survived'))
}, index=val_encoded.index)

In [33]:
# how good is our first model?
accuracy_score(y_pred.y_true, y_pred.dt_one)

0.8146067415730337

In [37]:
# we have approximately 81% accuracy on our first model
# compared to what baseline said. Its doing something! Huzah!
accuracy_score(y_val_pred.y_true, y_val_pred.dt_one)

0.7865168539325843

Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [46]:
clf.score(X_train, y_train)

0.8146067415730337

In [47]:
clf_cross = pd.crosstab(y_train, y_pred.dt_one)
clf_cross

dt_one,0,1
survived,,
0,293,36
1,63,142


In [48]:
print(classification_report(y_train, y_pred.dt_one))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86       329
           1       0.80      0.69      0.74       205

    accuracy                           0.81       534
   macro avg       0.81      0.79      0.80       534
weighted avg       0.81      0.81      0.81       534



Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [49]:
tp = clf_cross.iloc[0,0] #rows, columns
fn = clf_cross.iloc[0,1]
fp = clf_cross.iloc[1,0]
tn = clf_cross.iloc[1,1]

In [50]:
(tp + tn)/(tp + tn + fp + fn)

0.8146067415730337

Run through steps 2-4 using a different max_depth value.

In [55]:
clf2 = DecisionTreeClassifier(max_depth=8, min_samples_leaf=10)

In [56]:
clf2

DecisionTreeClassifier(max_depth=8, min_samples_leaf=10)

In [57]:
# fit the thing
# NOTE: ONLY DO THIS STEP ON TRAIN.
clf2.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=10)

In [58]:
# use the thing!
# now that clf is fit, use it to predict!
clf.predict(X_train)[:10]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

In [59]:
clf2.score(X_train, y_train)

0.8445692883895131

In [62]:
y_val_pred = pd.DataFrame({
    'y_true': val_encoded.survived,
    'baseline':0,
    'dt_one': clf2.predict(val_encoded.drop(columns='survived'))
}, index=val_encoded.index)

In [64]:
clf2_cross = pd.crosstab(y_train, y_pred.dt_one)
clf2_cross

dt_one,0,1
survived,,
0,293,36
1,63,142


In [65]:
print(classification_report(y_train, y_pred.dt_one))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86       329
           1       0.80      0.69      0.74       205

    accuracy                           0.81       534
   macro avg       0.81      0.79      0.80       534
weighted avg       0.81      0.81      0.81       534



Which model performs better on your in-sample data?

In [66]:
#clf2 performed better

Which model performs best on your out-of-sample data, the validate set?

In [79]:
clf.score(X_val, y_val)

0.7865168539325843

In [77]:
clf2.score(X_val, y_val)

0.7247191011235955

In [80]:
#clf works better